In [1]:
import nltk
from nltk.stem.snowball import FinnishStemmer

In [2]:
stemmer = FinnishStemmer()

In [ ]:
stemmer.stem("masturboinn")

'masturboin'

In [ ]:
from pyomorfi import load_omorfi
from pyomorfi.omorfi.analysis import Analysis

omorfi = load_omorfi()

analyses: list[Analysis] = omorfi.analyse("kissoja")
for a in analyses:
    print(a.lemmas)
    

In [35]:
import json
with open('test.json', 'r') as f:
    data = json.load(f)

segments = []
for seg in data['segments']:
    segments.append({
        "start": seg["start"],
        "end": seg["end"],
        "text": seg["text"],
    })

from pprint import pprint
pprint(segments)

[{'end': 29.0,
  'start': 0.0,
  'text': ' Ensimmäisen jaksossa me ajateltiin puhua masturboinnista, koska '
          'meidän mielestä kaikki hyvä paneiminen lähtee siitä, että '
          'masturbointi on kunnossa, koska mehän sama kuin sanotaan, että voi '
          'rakastaa muita ennen kuin osat rakastaa itseäsi tai saati '
          'vastaanottaa muilta rakkautta, niin vähän sama, että jos sä et '
          'pysty itselle tuottaa nautintoa, niin pystytkö sä sitä antamaan '
          'muille tai vastaanottamaan myöskään muilta, eli kaikki lähtee siitä '
          'omasta itsestään.'},
 {'end': 44.0,
  'start': 29.0,
  'text': ' Kyllä, ja itse ainakin suhtaudun masturbaatioon, kuullosti pääsä '
          'nyt tietenkin viralliseltaan se masturbaatioon, siis tumputukseen '
          'ihan yhtenä tärkeänä osana omaa hyvinvointia ja terveyttä.'}]


In [41]:
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')

word_tokenize("hello there")

[nltk_data] Downloading package punkt_tab to /home/matti/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


['hello', 'there']

In [45]:

def lemmatize_finnish(word):
    analysis: list[Analysis] = omorfi.analyse(word)
    if analysis == []:
        return ""
    return analysis[0].lemmas[0]

i = 0
for seg in segments:
    print('SEGMENT START:', seg['start'])
    text = seg['text']
    for word in word_tokenize(text.lower()):
        # if word != "":
        if word.isalpha():
            i += 1
            # print(f"  {i} {word}")
            lemma = lemmatize_finnish(word)
            print(lemma)

SEGMENT START: 0.0
ensimmäinen
jakso
me
ajatella
puhua
masturbointi
koska
me
mieli
kaikki
hyvä
paneiminen
lähteä
se
että
masturbointi
olla
kunto
koska
me
sama
kuu
san
että
voi
rakastaa
muu
ennen
kuu
osa
rakastaa
itse
tai
saati
vastaanottaa
muu
rakkaus
ne
vähä
sama
että
jos
sä
ei
pystyä
itse
tuottaa
nautinto
ne
pystyä
sä
se
antaa
muu
tai
vastaanottaa
myös
muu
elää
kaikki
lähteä
se
oma
itse
SEGMENT START: 29.0
kyllä
ja
itse
aa
suhta
masturbaatio
kuullosti
pääsä
nyt
tieten
virallinen
se
masturbaatio
siis
tumputukseen
ihan
yksi
tärkeä
osa
oma
hyvinvointi
ja
terveys


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

# Load Finnish BERT
model_name = "TurkuNLP/bert-base-finnish-cased-v1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def embed_sentences(sents: list[str]) -> np.ndarray:
    if not isinstance(sents, list):
        sents = [sents]
    inputs = tokenizer(sents, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)
        return embeddings.cpu().numpy()

/home/matti/.exe/himocast_downloader/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Example text
text = "Koirilla on pentuja."

# Tokenize
inputs = tokenizer(text, return_tensors="pt")

# Get embeddings
with torch.no_grad():
    outputs = model(**inputs)
    # outputs.last_hidden_state shape: [1, tokens, hidden_size]
    embeddings = outputs.last_hidden_state.mean(dim=1)  # sentence embedding


{'input_ids': tensor([[  102,  3227,  5786,   145, 30908,   111,   103]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
from nltk.tokenize import sent_tokenize

text = ""
with open('suomi_text.txt', 'r') as f:
    for line in f:
        if line != "\n":
            text += line.replace("\n", "").strip() + " "
for sent in sent_tokenize(text):
    print(sent)

In [48]:
# get embeddings

import numpy as np
sents = sent_tokenize(text)
sent_embeddings = np.zeros((len(sents), model.config.hidden_size), dtype="float32")
batch_size = 16

for i in range(0, len(sents), batch_size):
    print('i:', i)
    batch = sents[i : i+batch_size]
    print(len(batch))
    batch_embeds = embed_sentences(batch)
    print(batch_embeds.shape)
    print(i, i+batch_size)
    # print(sent_embeddings.shape, batch_embeds.shape)
    sent_embeddings[i:i+batch_size, :] = batch_embeds


# for idx, sent in enumerate(sentences):
#     print(" {:>5}/{}  \"{}\"".format(idx+1, len(sentences), sent[:160]))
#     embeds = embed_sentences(sent)
#     embeddings[idx, :] = embeds[0]

i: 0
16
(16, 768)
0 16
i: 16
16
(16, 768)
16 32
i: 32
16
(16, 768)
32 48
i: 48
16
(16, 768)
48 64
i: 64
16
(16, 768)
64 80
i: 80
16
(16, 768)
80 96
i: 96
16
(16, 768)
96 112
i: 112
16
(16, 768)
112 128
i: 128
16
(16, 768)
128 144
i: 144
15
(15, 768)
144 160


In [46]:
a = np.zeros((8, 4))
b = np.ones((3, 4))
a[5:10, :] = b
a.shape, b.shape
a

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]])

In [ ]:
a = np.zeros((3, 5))
a[1,:] = np.arange(5).reshape(1, 5)
a
np.arange(5).reshape(1, 5)

array([[0., 0., 0., 0., 0.],
       [0., 1., 2., 3., 4.],
       [0., 0., 0., 0., 0.]])

In [97]:
from sklearn.metrics.pairwise import cosine_similarity

text = "Suomi oli venäjän vallan alla"

query_emb = embed_sentences(text)
sims = cosine_similarity(sent_embeddings, query_emb).squeeze()
sorted_idx = np.flip( np.argsort(sims, axis=0) )[:100].squeeze()
pairs = np.array((sorted_idx, sims[sorted_idx])).T

for idx, sim in pairs[:5]:
    sent = sents[int(idx)]
    print(f"{sim:.4f}  |  {sent}")

0.7724  |  Tämän vuoksi Suomi oli usein Ruotsin ja venäläisten valtioiden (Novgorodin, Moskovan ja keisarillisen Venäjän) välisten sotien taistelutantereena tai sodankäynnin tukialueena.
0.7683  |  Venäjän vallan aika oli Suomen kannalta yksi historian rauhallisimmista kausista (vain Oolannin sota kosketti Suomen aluetta), mutta jonkin verran suomalaisia vapaaehtoisjoukkoja tai upseereita taisteli Venäjän lipun alla Manner-Euroopassa ja Aasiassa.
0.7600  |  Suomi, Baltian maat ja puolet Puolasta kuuluivat Neuvostoliiton etupiiriin.
0.7425  |  Suomen sotahistorian pitkäaikaisena piirteenä oli kuuluminen Ruotsin valtakuntaan keskiajalta vuoteen 1809 asti.
0.7398  |  Vuosina 1809–1917 Suomen suuriruhtinaskunnan sotahistoria oli luonnollisesti sidoksissa Venäjän keisarikuntaan.


In [92]:
a = np.array([
    [1,2,3,4],
    [4,1,4,2],
    [9,2,6,7],
    [0,0,1,0],
])
b = np.array([[2,2,3,2]])
sims = cosine_similarity(a, b).squeeze()
sorted_idx = np.flip( np.argsort(sims, axis=0) )[:3].squeeze()
pairs = np.array((sorted_idx, sims[sorted_idx])).T
# np.concatenate((sorted_idx, top_sims), axis=0)
pairs[0][0]

np.float64(1.0)